***Import Libraries And Noise Types Defined***

In [11]:
import numpy as np
import os
import random
from pydub import AudioSegment


noise_class_dictionary = {
0 : "air_conditioner",
1 : "car_horn",
2 : "children_playing",
3 : "dog_bark",
4 : "drilling",
5 : "engine_idling",
6 : "gun_shot",
7 : "jackhammer",
8 : "siren",
9 : "street_music"
}

noise_class_dictionary


{0: 'air_conditioner',
 1: 'car_horn',
 2: 'children_playing',
 3: 'dog_bark',
 4: 'drilling',
 5: 'engine_idling',
 6: 'gun_shot',
 7: 'jackhammer',
 8: 'siren',
 9: 'street_music'}

In [12]:

dataset="Datasets/"
urbanData="UrbanSound8K/"

fold_names = []
for i in range(1,11):
    fold_names.append("fold"+str(i)+"/")

fold_names

['fold1/',
 'fold2/',
 'fold3/',
 'fold4/',
 'fold5/',
 'fold6/',
 'fold7/',
 'fold8/',
 'fold9/',
 'fold10/']

In [13]:
Urban8Kdir =urbanData
target_folder = dataset+"clean_trainset_28spk_wav"


***Get Files With Different Noise Type***

In [14]:
def diffNoiseType(files,noise_type):
    result = []
    for i in files:
        if i.endswith(".wav"):
            fname = i.split("-")
            if fname[1] != str(noise_type):
                result.append(i)
    return result


***Files With Same Noise Type***

In [15]:
def oneNoiseType(files, noise_type):
    result = []
    for i in files:
        if i.endswith(".wav"):
            fname = i.split("-")
            if fname[1] == str(noise_type):
                result.append(i)
    return result

***Generate Noisy File From Clean File***

In [16]:
# def genNoise(filename, num_per_fold, dest):
#     clean_audio_path = target_folder+"/"+filename
#     audio_1 = AudioSegment.from_file(clean_audio_path)
#     counter = 0
#     for fold in fold_names:
#         dirname = Urban8Kdir + fold
#         # print(dirname)
#         dirlist = os.listdir(dirname)
#         total_noise_len = len(dirlist)
#         samples = np.random.choice(total_noise_len, num_per_fold, replace=False)
#         print(samples)
#         for s in samples:
#             noisefile = dirlist[s]
#             try:
#                 audio_2 = AudioSegment.from_file(dirname+"/"+noisefile)
#                 combined = audio_1.overlay(audio_2, times=5)
#                 target_dest = dest+"/"+filename[:len(filename)-4]+"_noise_"+str(counter)+".wav"
#                 combined.export(target_dest, format="wav")
#                 audio = Audio(target_dest)
#                 display(audio)
#                 counter +=1
#             except:
#                 print("Error")



In [17]:
# genNoise("p226_001.wav",1,"Datasets")

In [18]:
import torchaudio
from IPython.display import Audio

In [19]:
def genNoisyFile(filename,dest, noise_type,snr,isdiff):
    succ = False
    true_path = target_folder+"/"+filename
    count=0
    while not succ:
        try:
            audio_1 = AudioSegment.from_file(true_path)
        except:
            print("Some kind of audio decoding error occurred for base file... skipping")
            break

        un_noised_file, _ = torchaudio.load(true_path)
        un_noised_file = un_noised_file.numpy()
        un_noised_file = np.reshape(un_noised_file, -1)
        # Create an audio Power array
        un_noised_file_watts = un_noised_file ** 2
        # Create an audio Decibal array
        un_noised_file_db = 10 * np.log10(un_noised_file_watts)
        # Calculate signal power and convert to dB
        un_noised_file_avg_watts = np.mean(un_noised_file_watts)
        un_noised_file_avg_db = 10 * np.log10(un_noised_file_avg_watts)
        # Calculate noise power
        added_noise_avg_db = un_noised_file_avg_db - snr

        try:
            fold =  np.random.choice(fold_names, 1, replace=False)
            fold = fold[0]
            dirname = Urban8Kdir + fold
            dirlist = os.listdir(dirname)
            noisefile=""
            if isdiff:
                possible_noises=diffNoiseType(dirlist,noise_type)
                total_noise = len(possible_noises)
                samples = np.random.choice(total_noise, 1, replace=False)
                s = samples[0]
                noisefile = possible_noises[s] 
            else:
               possible_noises=oneNoiseType(dirlist,noise_type)
               total_noise = len(possible_noises)
               samples = np.random.choice(total_noise, 1, replace=False)
               s = samples[0]
               noisefile = possible_noises[s]


            noise_src_file, _ = torchaudio.load(dirname+"/"+noisefile)
            noise_src_file = noise_src_file.numpy()
            noise_src_file = np.reshape(noise_src_file, -1)
            noise_src_file_watts = noise_src_file ** 2
            noise_src_file_db = 10 * np.log10(noise_src_file_watts)
            noise_src_file_avg_watts = np.mean(noise_src_file_watts)
            noise_src_file_avg_db = 10 * np.log10(noise_src_file_avg_watts)
            
            db_change = added_noise_avg_db - noise_src_file_avg_db

            audio_2 = AudioSegment.from_file(dirname+"/"+noisefile)
            audio_2=audio_2+db_change
            combined = audio_1.overlay(audio_2, times=5)
            target_dest = dest+"/"+filename
            combined.export(target_dest, format="wav")
            # audio =Audio(target_dest)
            # display(audio)
            succ = True
        except:
            if count>5:
                succ=True
            count+=1
            pass
            # print("Some kind of audio decoding error occurred for the noise file..retrying")
            # break;



In [20]:
noise_type = int(input("Enter the noise class dataset to generate :\t"))

In [21]:
inp_folder = dataset+"US_Class3"+str(noise_type)+"_Train_Input"
op_folder = dataset+"US_Class3"+str(noise_type)+"_Train_Output"

In [22]:
print("Generating Training Data..")
print("Making train input folder")
if not os.path.exists(inp_folder):
    os.makedirs(inp_folder)
print("Making train output folder")
if not os.path.exists(op_folder):
    os.makedirs(op_folder)

Generating Training Data..
Making train input folder
Making train output folder


In [23]:
from tqdm import tqdm
print(os.listdir(target_folder).__len__())

11574


In [24]:
def generate_Train_dataset():
    counter = 0
    #noise_type = 1
    for file in tqdm(os.listdir(target_folder)):
        filename = os.fsdecode(file)
        if filename.endswith(".wav"):
            snr = random.randint(0,10)
            # noise_type=random.randint(0,9)
            genNoisyFile(filename,inp_folder,noise_type,snr,0)
            snr = random.randint(0,10)
            genNoisyFile(filename,op_folder,noise_type,snr,1)
            counter +=1
        if counter>=100:
            break



In [25]:
generate_Train_dataset()

  0%|          | 0/11574 [00:00<?, ?it/s]C:\Users\Avnesh\AppData\Local\Temp\ipykernel_3388\2593736498.py:18: RuntimeWarning: divide by zero encountered in log10
  un_noised_file_db = 10 * np.log10(un_noised_file_watts)
C:\Users\Avnesh\AppData\Local\Temp\ipykernel_3388\2593736498.py:49: RuntimeWarning: divide by zero encountered in log10
  noise_src_file_db = 10 * np.log10(noise_src_file_watts)
  0%|          | 20/11574 [00:01<18:23, 10.47it/s]c:\Users\Avnesh\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydub\utils.py:198: RuntimeWarning: Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work
  warn("Couldn't find ffprobe or avprobe - defaulting to ffprobe, but may not work", RuntimeWarning)
  1%|          | 100/11574 [00:07<14:42, 13.01it/s]


In [26]:
Urban8Kdir = urbanData
target_folder =  dataset+"clean_testset_wav"
inp_folder = dataset+"US_Class3"+str(noise_type)+"_Test_Input"
print(os.listdir(target_folder).__len__())

100


In [27]:
print("Generating Testing Data..")
print("Making test input folder")
if not os.path.exists(inp_folder):
    os.makedirs(inp_folder)


Generating Testing Data..
Making test input folder


In [28]:
def generate_Test_dataset():
    counter = 0
    for file in tqdm(os.listdir(target_folder)):
        filename = os.fsdecode(file)
        if filename.endswith(".wav"):
            snr = random.randint(0,10)
            genNoisyFile(filename,inp_folder,noise_type,snr,0)
            counter +=1
        if counter>=100:
            break

In [29]:
generate_Test_dataset()

  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\Avnesh\AppData\Local\Temp\ipykernel_3388\2593736498.py:18: RuntimeWarning: divide by zero encountered in log10
  un_noised_file_db = 10 * np.log10(un_noised_file_watts)
C:\Users\Avnesh\AppData\Local\Temp\ipykernel_3388\2593736498.py:49: RuntimeWarning: divide by zero encountered in log10
  noise_src_file_db = 10 * np.log10(noise_src_file_watts)
 99%|█████████▉| 99/100 [00:03<00:00, 31.86it/s]
